In [ ]:
from hypothesis import strategies as st
from octo_spork.expressions import Attribute, Le, Lt, Ge, Gt, And, Or, Not

def _link_clauses(clauses):
    linked = st.one_of(
        st.lists(clauses, min_size=1).map(And),
        st.lists(clauses, min_size=1).map(Or))
    return linked | linked.map(Not)

def inequalities(attributes, values):
    ''' Combines names and values to create an inequality clause strategy. '''
    return st.one_of(
        st.tuples(attributes, values).map(lambda r: Le(*r)),
        st.tuples(attributes, values).map(lambda r: Lt(*r)),
        st.tuples(attributes, values).map(lambda r: Ge(*r)),
        st.tuples(attributes, values).map(lambda r: Gt(*r)))

def attributes(names):
    ''' Draw from the given names to create a strategy for Attribute
    objects. If names is an iterable it will be converted to a one_of,
    otherwise names are used directly.'''
    try:
        names = st.one_of(st.just(n) for n in names)
    except:
        pass
    return names.map(Attribute)

def expressions(clauses, max_leaves):
    ''' Generate compound expressions from the given clause strategy,
    linking the tree structure with And/Or/Not branches. '''
    return st.recursive(clauses, _link_clauses, max_leaves)

In [ ]:
clauses = inequalities(attributes('xyz'), values=st.integers(min_value=-10, max_value=10))
strategy = expressions(clauses, max_leaves=20)

In [ ]:
from octo_spork.logic import to_dnf, to_cnf
from octo_spork.simplify import simplify

expression = strategy.example()
print(expression)
print(simplify(to_dnf(simplify(expression))))